### Import file and library

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!pip install ase
import os
import sys
import re
import copy
import string
import math
import numpy as np
import pandas as pd
from ase.io import read

from operator import itemgetter
from itertools import groupby
!pip install pubchempy
import pubchempy as pcp

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import AllChem, Draw, rdDepictor
from rdkit.Chem import Lipinski
from rdkit import DataStructs
from rdkit import *


--2021-10-12 10:07:43--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.20’

Miniconda3-py37_4.8 100%[===================>]  81.12M  66.9MB/s    in 1.2s    

2021-10-12 10:07:44 (66.9 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.20’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7

In [ ]:
train = pd.read_csv('trainSurfaceVsa.csv')
train.head()


,MOFname,volume [A^3],weight [u],density [g/cm^3],surface_area [m^2/g],gsa,vsa,void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,1.301526,1139.828774,585.669052,1139.828774,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
1,mof_unit_2,2769.503842,2211.697211,1.326090,603.610000,182.839127,477.993349,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
2,mof_unit_3,1089.818728,773.687960,1.178856,788.500000,1704.599832,1614.212392,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
3,mof_unit_4,2205.198301,1304.638720,0.982408,1441.530000,3925.613160,1911.564582,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
4,mof_unit_5,1137.800963,901.736120,1.316020,277.736656,197.658635,277.736656,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001


In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    train[col] = train[col].astype('category')
train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68613 entries, 0 to 68612
Data columns (total 17 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   MOFname                                        68613 non-null  object  
 1   volume [A^3]                                   68613 non-null  float64 
 2   weight [u]                                     68613 non-null  float64 
 3   density [g/cm^3]                               68613 non-null  float64 
 4   surface_area [m^2/g]                           68613 non-null  float64 
 5   gsa                                            68613 non-null  float64 
 6   vsa                                            68613 non-null  float64 
 7   void_fraction                                  65683 non-null  float64 
 8   void_volume [cm^3/g]                           65483 non-null  float64 
 9   functional_groups                      

In [ ]:
train.dropna(inplace = True)
train.isnull().sum()

MOFname                                          0
volume [A^3]                                     0
weight [u]                                       0
density [g/cm^3]                                 0
surface_area [m^2/g]                             0
gsa                                              0
vsa                                              0
void_fraction                                    0
void_volume [cm^3/g]                             0
functional_groups                                0
metal_linker                                     0
organic_linker1                                  0
organic_linker2                                  0
topology                                         0
CO2/N2_selectivity                               0
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    0
CO2_working_capacity [mL/g]                      0
dtype: int64

In [ ]:
train.shape

(65138, 17)

In [ ]:
#train = train[train['functional_groups'] !='H']

## Get smiles

In [ ]:
def split_formula(formula):
    """เพื่อแยกตัวประกอบจาก functional groups """
    list_chem = []
    list_formula = formula.split("-")
    n = len(list_formula)
    for f in list_formula:
        for i in range(len(f)):
            if f[i].isupper() == False :
               new_f = f[i-1]+f[i]
               list_chem.append(new_f)
            else:
                try:
                    if f[i+1].isupper() == True:                
                        list_chem.append(f[i])
                except:
                    list_chem.append(f[i])
                else :
                    pass

    return list_chem

def search_dict(dict_,func,num):
    found = "0"
    for k,v in dict_.items():
        if k == func:
            found = v[num]
        
    return found

def remove_h(func_group):
    try:
        group = re.search("H[0-9]*",func_group).group()
        
        func_group = func_group.replace(group,"")
        
    except:
        pass
    return func_group

def inside_algorithm(func,num,list_):
    if search_dict(smiles_dict,func,0) == '0':
            func = remove_h(func) #เอา h ออก
            m = split_formula(func) # split ที่เหลือ
            for i in range(len(m)):
                if i == 0 :
                    list_.append(search_dict(smiles_dict,m[i],0))
                else :list_.append(search_dict(smiles_dict,m[i],1))
    else: list_.append(smiles_dict[func][num-1])

def convert_to_smiles(smiles_dict,func_group):
    smiles=""
    get_smiles = []
    #splitด้วย- ขั้นตอนต่อมา เอาที่สปลิตได้ทีละตัวไปค้นหาว่ามีอยู่ในsmiles dict มั้ย ถ้ามี แทนค่า การแทนค่า ถ้าอยู่ตัวแรกใช่valueตัวที่1 ถ้าอยู่ตัวที่2ใช้valueตัวที่2 ถ้าไม่มีให้ใช้ฟังก์ชั่น split formula แยกแล้วค้นหาในdictอีกที แล้วแทนค่า ขั้นตอนสุดท้าย เอาที่แทนค่าได้ทั้งหมดมาต่อกัน
    func_split = func_group.split('-')
    # ครั้งที่ 1
    inside_algorithm(func_split[0],1,get_smiles)
    #ครั้งที่ 2 
    try :
        inside_algorithm(func_split[1],2,get_smiles)
    except: pass
    for word in get_smiles:
        smiles+=word
    
    return smiles,smiles.replace("-","")


In [ ]:
#molecule to smiles
smiles_dict = {
    "N":["N","-N"],
    "F":["F-","-F"],
    "Cl":["Cl-","-Cl"],
    "Br":["Br-","-Br"],
    "I":["I-","-I"],
    "Me":["C-","-C"],
    "Et":["CC-","-CC"],
    "Pr":["CCC-","-CCC"],
    "CO":["C(=O)-","-C(=O)"],
    "COOH":["OC(=O)-","-C(=O)O"],
    "OH":["O-","-O"],
    "OMe":["CO-","-OC"],
    "OEt":["CCO-","-OCC"],
    "OPr":["CCCO-","-OCCC"],
    "NH2":["N-","-N"],
    "CN":["C(#N)-","-C(#N)"],
    "NHCH3":["CN-","-NC"],
    "NO2":["(O=)(O=)N-","-N(=O)(=O)"],
    "Ph":["C1=CC=CC=C1-","-C1=CC=CC=C1-"],
    "SO2OH":["OS(=O)(=O)-","-S(=O)(=O)O"],
    "SO3H":["OS(=O)(=O)-","-S(=O)(=O)O"],
     "HCO" : ["C(=O)-" ,"-C(=O)"],
    "NHMe":["CN-","-NC"]
}

In [ ]:
NO2_dict = {"H":"[H]",
    'Pr-NO2':'CCC[N+](=O)[O-]',
'NHMe-NO2' :'CN[N+](=O)([O-])', 
'NO2-NH2' : '[N+](=O)([O-])N',
'NO2-OH' :'[N+](=O)([O-])O',
'NO2-I' :  '[N+](=O)([O-])I',
'NO2-HCO' :'[N+](=O)([O-])C(=O)',
'OMe-NO2': 'CO[N+](=O)[O-]',
 'NO2': '[N+](=O)([O-])',
 'NO2-Ph': '[N+](=O)([O-])C1=C(C=CC=C1)',
 'OPr-NO2': 'CCCO[N+](=O)([O-])',
 'Me-NO2': 'C[N+](=O)([O-])',
 'NO2-F': '[N+](=O)([O-])F',
 'Cl-NO2': 'Cl[N+](=O)([O-])',
 'NO2-SO3H': '[N+](=O)([O-])S(=O)(=O)O',
'NH2-NO2': 'N[N+](=O)([O-])',
 'NO2-OMe': '[N+](=O)([O-])OC',
 'F-NO2': 'F[N+](=O)([O-])',
 'NO2-Me': '[N+](=O)([O-])C',
 'CN-NO2': 'C(#N)[N+](=O)([O-])',
 'Br-NO2': 'Br[N+](=O)([O-])',
 'NO2-Br': '[N+](=O)([O-])Br',
 'NO2-CN': '[N+](=O)([O-])C(#N)',
 'OH-NO2': 'O[N+](=O)([O-])',
 'NO2-Pr': '[N+](=O)([O-])CCC',
 'H-NO2':'[N+](=O)([O-])',
 'NO2-NHMe': '[N+](=O)([O-])NC',
 'NO2-OPr':'[N+](=O)([O-])OCCC',
 'Ph-NO2':'C1=C(C=CC=C1)[N+](=O)([O-])',
 'NO2-Cl': '[N+](=O)([O-])Cl',
 'NO2-H':'[N+](=O)([O-])',
 'I-NO2': 'I[N+](=O)([O-])',
 'HCO-NO2': 'C(=O)[N+](=O)([O-])',
 'SO3H-NO2': 'OS(=O)(=O)[N+](=O)([O-])',
 'COOH-NO2': 'OC(=O)[N+](=O)([O-])',
 'NO2-COOH': '[N+](=O)([O-])C(=O)O',
 'NO2-Et': '[N+](=O)([O-])CC',
 'NO2-OEt':'[N+](=O)([O-])OCC',
 'Et-NO2': 'CC[N+](=O)([O-])',
 'OEt-NO2': 'CCO[N+](=O)([O-])'}

In [ ]:
formula_list = train['functional_groups'].unique()
smiles_list = [convert_to_smiles(smiles_dict,x)[1] for x in formula_list]

In [ ]:
for i,formula in enumerate(formula_list):
    for k,v in NO2_dict.items():
        if formula == k:
            smiles_list[i] = v


In [ ]:
functional_groups_dict = dict(zip(formula_list, smiles_list))

In [ ]:
functional_groups_dict

{'COOH-OEt': 'OC(=O)OCC',
 'F-OMe': 'FOC',
 'OMe-COOH': 'COC(=O)O',
 'H-SO3H': 'S(=O)(=O)O',
 'NHMe-OH': 'CNO',
 'Pr-NO2': 'CCC[N+](=O)[O-]',
 'NH2': 'N',
 'Br': 'Br',
 'HCO': 'C(=O)',
 'SO3H': 'OS(=O)(=O)',
 'Me-Br': 'CBr',
 'NHMe-NO2': 'CN[N+](=O)([O-])',
 'OEt-CN': 'CCOC(#N)',
 'OMe-NH2': 'CON',
 'Cl-Pr': 'ClCCC',
 'Me-F': 'CF',
 'SO3H-Pr': 'OS(=O)(=O)CCC',
 'CN': 'C(#N)',
 'H-Cl': 'Cl',
 'COOH-OH': 'OC(=O)O',
 'Et-OH': 'CCO',
 'OH-Pr': 'OCCC',
 'SO3H-COOH': 'OS(=O)(=O)C(=O)O',
 'OEt-H': 'CCO',
 'SO3H-OEt': 'OS(=O)(=O)OCC',
 'OMe': 'CO',
 'Et-Br': 'CCBr',
 'OH-F': 'OF',
 'NO2-NH2': '[N+](=O)([O-])N',
 'NHMe-Me': 'CNC',
 'OPr-SO3H': 'CCCOS(=O)(=O)O',
 'Ph-Me': 'C1=CC=CC=C1C',
 'Me': 'C',
 'OPr-Br': 'CCCOBr',
 'COOH-Me': 'OC(=O)C',
 'F': 'F',
 'NO2-OH': '[N+](=O)([O-])O',
 'Ph': 'C1=CC=CC=C1',
 'OH': 'O',
 'HCO-SO3H': 'C(=O)S(=O)(=O)O',
 'Cl-HCO': 'ClC(=O)',
 'OPr': 'CCCO',
 'NHMe': 'CN',
 'H-COOH': 'C(=O)O',
 'Cl-OPr': 'ClOCCC',
 'COOH-Cl': 'OC(=O)Cl',
 'NH2-OMe': 'NOC',
 'Pr-SO3H': 

In [ ]:
smiles =[functional_groups_dict[formula] for formula in train['functional_groups']]
train.insert(7, 'smiles', smiles)

In [ ]:
train.head()

,MOFname,volume [A^3],weight [u],density [g/cm^3],surface_area [m^2/g],gsa,vsa,smiles,void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,1.301526,1139.828774,585.669052,1139.828774,OC(=O)OCC,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
1,mof_unit_2,2769.503842,2211.697211,1.326090,603.610000,182.839127,477.993349,FOC,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
2,mof_unit_3,1089.818728,773.687960,1.178856,788.500000,1704.599832,1614.212392,COC(=O)O,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
3,mof_unit_4,2205.198301,1304.638720,0.982408,1441.530000,3925.613160,1911.564582,S(=O)(=O)O,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
4,mof_unit_5,1137.800963,901.736120,1.316020,277.736656,197.658635,277.736656,CNO,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001


## Get feature set 1 from smiles

In [ ]:
MolWt  = [Descriptors.MolWt(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
MolLogP = [Descriptors.MolLogP(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumValenceElectrons= [Descriptors.NumValenceElectrons(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
HeavyAtomCount= [Lipinski.HeavyAtomCount(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NOCount= [Lipinski.NOCount(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumAliphaticRings= [Lipinski.NumAliphaticRings(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumAromaticRings= [Lipinski.NumAromaticRings(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumHAcceptors= [Lipinski.NumHAcceptors(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumHDonors= [Lipinski.NumHDonors(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumHeteroatoms= [Lipinski.NumHeteroatoms(Chem.MolFromSmiles(mol)) for mol in train['smiles']]
NumRotatableBonds= [Lipinski.NumRotatableBonds(Chem.MolFromSmiles(mol)) for mol in train['smiles']]


[10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
[10:17:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:17:53] WARNING: not removing hydrogen atom wit

In [ ]:
list_of_f_ = ['MolWt','MolLogP','NumValenceElectrons','HeavyAtomCount','NOCount','NumAliphaticRings','NumAromaticRings','NumHAcceptors','NumHDonors','NumHeteroatoms','NumRotatableBonds']
list_dfset = [MolWt,MolLogP,NumValenceElectrons,HeavyAtomCount,NOCount,NumAliphaticRings,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds]
for i in range(len(list_of_f_)):
    train.insert(8+i,list_of_f_[i],list_dfset[i])


In [ ]:
train.head()

,MOFname,volume [A^3],weight [u],density [g/cm^3],surface_area [m^2/g],gsa,vsa,smiles,MolWt,MolLogP,...,void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,mof_unit_1,1116.667429,875.240600,1.301526,1139.828774,585.669052,1139.828774,OC(=O)OCC,90.078,0.7009,...,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
1,mof_unit_2,2769.503842,2211.697211,1.326090,603.610000,182.839127,477.993349,FOC,50.032,0.5173,...,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
2,mof_unit_3,1089.818728,773.687960,1.178856,788.500000,1704.599832,1614.212392,COC(=O)O,76.051,0.3108,...,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
3,mof_unit_4,2205.198301,1304.638720,0.982408,1441.530000,3925.613160,1911.564582,S(=O)(=O)O,82.080,-0.9292,...,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
4,mof_unit_5,1137.800963,901.736120,1.316020,277.736656,197.658635,277.736656,CNO,47.057,-0.4050,...,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001


In [ ]:
train.isnull().sum()

MOFname                                          0
volume [A^3]                                     0
weight [u]                                       0
density [g/cm^3]                                 0
surface_area [m^2/g]                             0
gsa                                              0
vsa                                              0
smiles                                           0
MolWt                                            0
MolLogP                                          0
NumValenceElectrons                              0
HeavyAtomCount                                   0
NOCount                                          0
NumAliphaticRings                                0
NumAromaticRings                                 0
NumHAcceptors                                    0
NumHDonors                                       0
NumHeteroatoms                                   0
NumRotatableBonds                                0
void_fraction                  

In [ ]:
train[train['functional_groups'] == 'H']

,MOFname,volume [A^3],weight [u],density [g/cm^3],surface_area [m^2/g],gsa,vsa,smiles,MolWt,MolLogP,...,void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
132,mof_unit_133,2879.399691,1051.91172,0.606634,3781.03,16953.539411,3202.028632,[H],1.008,0.1125,...,0.44697,0.7368,H,2,4,29,pcu,8.721837,4.484787,46.937370
290,mof_unit_291,2044.232116,987.95388,0.802520,2295.55,3139.502280,2174.009434,[H],1.008,0.1125,...,0.30162,0.3758,H,2,9,18,pcu,19.283622,5.808452,179.050773
461,mof_unit_462,4361.881659,1536.86384,0.585074,3797.52,11136.446282,2975.027385,[H],1.008,0.1125,...,0.45851,0.7837,H,9,18,27,sra,25.312399,5.845265,146.852097
544,mof_unit_545,3889.091818,2517.23368,1.074793,966.09,2268.912592,2310.126482,[H],1.008,0.1125,...,0.17732,0.1650,H,4,11,14,acs,25.420338,5.802776,91.865759
845,mof_unit_846,3915.123327,2020.59120,0.857003,2303.80,2911.660229,2057.217937,[H],1.008,0.1125,...,0.36882,0.4304,H,2,41,41,fof,24.463281,6.055741,248.006219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66159,mof_unit_66160,23903.512323,3720.14056,0.258432,6078.12,87787.934345,2944.435425,[H],1.008,0.1125,...,0.76886,2.9751,H,1,6,23,pcu,3.247642,2.282497,-13.429472
66196,mof_unit_66197,16722.133301,3154.40896,0.313239,5480.76,59563.186579,3178.028563,[H],1.008,0.1125,...,0.72851,2.3257,H,3,2,5,nbo,4.123984,3.132897,-5.675992
66333,mof_unit_66334,3038.164911,986.20824,0.539022,4383.39,16072.527166,4297.785749,[H],1.008,0.1125,...,0.53633,0.9950,H,9,20,25,sra,5.179175,3.431512,-0.121001
66389,mof_unit_66390,4980.364066,1552.42764,0.517606,3468.47,13267.258994,3577.287129,[H],1.008,0.1125,...,0.55357,1.0695,H,2,22,24,nbo,3.631160,3.049430,-1.408763


## Get feature set2 from smiles

In [ ]:
props_df = pd.read_csv('props_df.csv')

In [ ]:
data = []
columns = ['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D']
for i,v in enumerate(smiles_list):
    props = pcp.get_properties(['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D'], v, 'smiles')
    print(i)
    data.append(props)

In [ ]:
rows = []
columns = data[0][0].keys()
for i in range(len(smiles_list)):
    rows.append(data[i][0].values())
props_df = pd.DataFrame(data=rows, columns=columns) 
props_df.head()

In [ ]:
props_df.insert(0, 'smiles', smiles_list)
props_df.head()

In [ ]:
#props_df.to_csv('props_df.csv')

In [ ]:
train_copy = train.copy()

In [ ]:
props_df = props_df.drop(columns = ['HeavyAtomCount','Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D'])

In [ ]:
cols = list(props_df.columns)
train_copy[cols[2:]] = np.nan


In [ ]:
props_df = props_df.drop(columns = 'Unnamed: 0')
props_df.head()

,smiles,CID,MolecularFormula,MolecularWeight,InChI,InChIKey,IUPACName,XLogP,ExactMass,MonoisotopicMass,...,HBondAcceptorCount,RotatableBondCount,IsotopeAtomCount,AtomStereoCount,DefinedAtomStereoCount,UndefinedAtomStereoCount,BondStereoCount,DefinedBondStereoCount,UndefinedBondStereoCount,CovalentUnitCount
0,OC(=O)OCC,105349,C3H6O3,90.080,"InChI=1S/C3H6O3/c1-2-6-3(4)5/h2H2,1H3,(H,4,5)",CQDGTJPVBWZJAZ-UHFFFAOYSA-N,ethyl hydrogen carbonate,0.6,90.031694,90.031694,...,3,2,0,0,0,0,0,0,0,1.0
1,FOC,142072,CH3FO,50.032,InChI=1S/CH3FO/c1-3-2/h1H3,XMSZANIMCDLNKA-UHFFFAOYSA-N,methyl hypofluorite,0.6,50.016793,50.016793,...,2,0,0,0,0,0,0,0,0,1.0
2,COC(=O)O,78579,C2H4O3,76.050,"InChI=1S/C2H4O3/c1-5-2(3)4/h1H3,(H,3,4)",CXHHBNMLPJOKQD-UHFFFAOYSA-N,methyl hydrogen carbonate,0.2,76.016044,76.016044,...,3,1,0,0,0,0,0,0,0,1.0
3,S(=O)(=O)O,104748,HO3S-,81.070,"InChI=1S/H2O3S/c1-4(2)3/h(H2,1,2,3)/p-1",LSNNMFCWUKXFEE-UHFFFAOYSA-M,hydrogen sulfite,-1.0,80.964640,80.964640,...,4,0,0,0,0,0,0,0,0,1.0
4,CNO,11647,CH5NO,47.057,"InChI=1S/CH5NO/c1-2-3/h2-3H,1H3",CPQCSJYYDADLCZ-UHFFFAOYSA-N,N-methylhydroxylamine,-0.7,47.037114,47.037114,...,2,0,0,0,0,0,0,0,0,1.0


In [ ]:
props_df = props_df.set_index('smiles')

In [ ]:
props_dict = props_df.to_dict()

In [ ]:
for col in cols[2:]:
    train_copy[col] = train_copy['smiles'].map(props_dict[col])


In [ ]:
nan_h = {
'CID':'5362549', 
'MolecularFormula':'H',
'MolecularWeight':'1.0080', 
'InChI':'InChI=1S/H', 
'InChIKey':'YZCKVEUIGOORGS-UHFFFAOYSA-N', 
'IUPACName':'hydrogen', 
'XLogP':'0',
'ExactMass':'1.0078250319', 
'MonoisotopicMass':'1.0078250319', 
'TPSA':'0', 
'Complexity':'0', 
'Charge':'0',
'HBondDonorCount':'0', 
'HBondAcceptorCount':'0', 
'RotatableBondCount':'0',
'IsotopeAtomCount':'0', 
'AtomStereoCount':'0', 
'DefinedAtomStereoCount':'0',
'UndefinedAtomStereoCount':'0', 
'BondStereoCount':'0', 
'DefinedBondStereoCount':'0',
'UndefinedBondStereoCount':'0', 
'CovalentUnitCount':'1'
}

In [ ]:
for k,v in nan_h.items():
    train_copy.loc[train_copy["functional_groups"] == 'H', k] = v

In [ ]:
train_copy.to_csv('train_final.csv',index= False)

# For test

In [ ]:
test_28 = pd.read_csv('test_28.csv')
test_28.shape

(17000, 13)

In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    test_28[col] = test_28[col].astype('category')
test_28.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 13 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   MOFname                                        17000 non-null  object  
 1   volume [A^3]                                   17000 non-null  float64 
 2   weight [u]                                     17000 non-null  float64 
 3   surface_area [m^2/g]                           17000 non-null  float64 
 4   void_fraction                                  17000 non-null  float64 
 5   void_volume [cm^3/g]                           17000 non-null  float64 
 6   functional_groups                              17000 non-null  object  
 7   metal_linker                                   17000 non-null  category
 8   organic_linker1                                17000 non-null  category
 9   organic_linker2                        

In [ ]:
smiles =[functional_groups_dict[formula] for formula in test_28['functional_groups']]
test_28.insert(7, 'smiles', smiles)

In [ ]:

MolWt  = [Descriptors.MolWt(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
MolLogP = [Descriptors.MolLogP(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumValenceElectrons= [Descriptors.NumValenceElectrons(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
HeavyAtomCount= [Lipinski.HeavyAtomCount(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NOCount= [Lipinski.NOCount(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumAliphaticRings= [Lipinski.NumAliphaticRings(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumAromaticRings= [Lipinski.NumAromaticRings(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumHAcceptors= [Lipinski.NumHAcceptors(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumHDonors= [Lipinski.NumHDonors(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumHeteroatoms= [Lipinski.NumHeteroatoms(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]
NumRotatableBonds= [Lipinski.NumRotatableBonds(Chem.MolFromSmiles(mol)) for mol in test_28['smiles']]

[11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom without neighbors
[11:19:11] WARNING: not removing hydrogen atom wit

In [ ]:
list_of_f_ = ['MolWt','MolLogP','NumValenceElectrons','HeavyAtomCount','NOCount','NumAliphaticRings','NumAromaticRings','NumHAcceptors','NumHDonors','NumHeteroatoms','NumRotatableBonds']
list_dfset = [MolWt,MolLogP,NumValenceElectrons,HeavyAtomCount,NOCount,NumAliphaticRings,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds]
for i in range(len(list_of_f_)):
    test_28.insert(8+i,list_of_f_[i],list_dfset[i])

In [ ]:
smiles_list = test_28['smiles'].unique()
len(smiles_list)

342

In [ ]:
data_ = []
columns = ['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D']
for i,v in enumerate(smiles_list):
    props = pcp.get_properties(['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D'], v, 'smiles')
    print(i)
    data_.append(props)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
rows = []
columns = data_[0][0].keys()
for i in range(len(smiles_list)):
    rows.append(data_[i][0].values())
props_df_ = pd.DataFrame(data=rows, columns=columns) 
props_df_.head()

,CID,MolecularFormula,MolecularWeight,InChI,InChIKey,IUPACName,XLogP,ExactMass,MonoisotopicMass,TPSA,...,FeatureCount3D,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D
0,702,C2H6O,46.07,"InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",LFQSCWFLJHTTHZ-UHFFFAOYSA-N,ethanol,-0.1,46.041864811,46.041864811,20.2,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
1,24841,HI,127.9124,InChI=1S/HI/h1H,XMBWDFGMSWQBCA-UHFFFAOYSA-N,iodane,0.9,127.91230,127.91230,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.0,1.0
2,540,CHNO,43.025,InChI=1S/CHNO/c2-1-3/h3H,XLJMAIOERFSOGZ-UHFFFAOYSA-N,cyanic acid,0.1,43.005813655,43.005813655,44.0,...,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
3,887,CH4O,32.042,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,methanol,-0.5,32.026214747,32.026214747,20.2,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
4,7903,C3H7NO2,89.09,"InChI=1S/C3H7NO2/c1-2-3-4(5)6/h2-3H2,1H3",JSZOAYXJRCEYSX-UHFFFAOYSA-N,1-nitropropane,0.9,89.047678466,89.047678466,45.8,...,4.0,2.0,0.0,1.0,0.0,0.0,1.0,0.4,2.0,8.0


In [ ]:
props_df_.insert(0, 'smiles', smiles_list)
props_df_.head()

,smiles,CID,MolecularFormula,MolecularWeight,InChI,InChIKey,IUPACName,XLogP,ExactMass,MonoisotopicMass,...,FeatureCount3D,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D
0,CCO,702,C2H6O,46.07,"InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",LFQSCWFLJHTTHZ-UHFFFAOYSA-N,ethanol,-0.1,46.041864811,46.041864811,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
1,I,24841,HI,127.9124,InChI=1S/HI/h1H,XMBWDFGMSWQBCA-UHFFFAOYSA-N,iodane,0.9,127.91230,127.91230,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.0,1.0
2,C(#N)O,540,CHNO,43.025,InChI=1S/CHNO/c2-1-3/h3H,XLJMAIOERFSOGZ-UHFFFAOYSA-N,cyanic acid,0.1,43.005813655,43.005813655,...,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
3,CO,887,CH4O,32.042,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,methanol,-0.5,32.026214747,32.026214747,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.4,0.0,1.0
4,[N+](=O)([O-])CCC,7903,C3H7NO2,89.09,"InChI=1S/C3H7NO2/c1-2-3-4(5)6/h2-3H2,1H3",JSZOAYXJRCEYSX-UHFFFAOYSA-N,1-nitropropane,0.9,89.047678466,89.047678466,...,4.0,2.0,0.0,1.0,0.0,0.0,1.0,0.4,2.0,8.0


In [ ]:
props_df_ =props_df_.set_index('smiles')

In [ ]:
props_df_.columns

Index(['CID', 'MolecularFormula', 'MolecularWeight', 'InChI', 'InChIKey',
       'IUPACName', 'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA',
       'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount',
       'RotatableBondCount', 'HeavyAtomCount', 'IsotopeAtomCount',
       'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount',
       'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount',
       'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D',
       'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D',
       'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D',
       'FeatureCationCount3D', 'FeatureRingCount3D',
       'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D',
       'EffectiveRotorCount3D', 'ConformerCount3D'],
      dtype='object')

In [ ]:
props_df_ = props_df_.drop(columns = ['HeavyAtomCount','Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D'])

In [ ]:
cols = list(props_df_.columns)
test_28[cols] = np.nan

In [ ]:
props_dict = props_df_.to_dict()

In [ ]:
for col in cols:
    test_28[col] = test_28['smiles'].map(props_dict[col])


In [ ]:
for k,v in nan_h.items():
    test_28.loc[test_28["functional_groups"] == 'H', k] = v

In [ ]:
test_28.isnull().sum()

MOFname                                          0
volume [A^3]                                     0
weight [u]                                       0
surface_area [m^2/g]                             0
void_fraction                                    0
void_volume [cm^3/g]                             0
functional_groups                                0
smiles                                           0
MolWt                                            0
MolLogP                                          0
NumValenceElectrons                              0
HeavyAtomCount                                   0
NOCount                                          0
NumAliphaticRings                                0
NumAromaticRings                                 0
NumHAcceptors                                    0
NumHDonors                                       0
NumHeteroatoms                                   0
NumRotatableBonds                                0
metal_linker                   

In [ ]:
set(train_copy.columns) - set(test_28.columns)

{'CO2_working_capacity [mL/g]', 'gsa', 'vsa'}

In [ ]:
test_28.insert(
    loc=3,
    column="density [g/cm^3]",
    value=(test_28["weight [u]"] / test_28["volume [A^3]"]) * 1.66054,
)

In [ ]:
l = list(train_copy.columns)
l.remove('CO2_working_capacity [mL/g]')
l.remove('gsa')
l.remove('vsa')

In [ ]:
test_28 = test_28[l]

In [ ]:
test_28.columns

Index(['MOFname', 'volume [A^3]', 'weight [u]', 'density [g/cm^3]',
       'surface_area [m^2/g]', 'smiles', 'MolWt', 'MolLogP',
       'NumValenceElectrons', 'HeavyAtomCount', 'NOCount', 'NumAliphaticRings',
       'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms',
       'NumRotatableBonds', 'void_fraction', 'void_volume [cm^3/g]',
       'functional_groups', 'metal_linker', 'organic_linker1',
       'organic_linker2', 'topology', 'CO2/N2_selectivity',
       'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]', 'CID',
       'MolecularFormula', 'MolecularWeight', 'InChI', 'InChIKey', 'IUPACName',
       'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity',
       'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount',
       'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount',
       'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount',
       'UndefinedBondStereoCount', 'CovalentUnitCount'],
      dtype='object'

In [ ]:
train_copy.columns

Index(['MOFname', 'volume [A^3]', 'weight [u]', 'density [g/cm^3]',
       'surface_area [m^2/g]', 'gsa', 'vsa', 'smiles', 'MolWt', 'MolLogP',
       'NumValenceElectrons', 'HeavyAtomCount', 'NOCount', 'NumAliphaticRings',
       'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms',
       'NumRotatableBonds', 'void_fraction', 'void_volume [cm^3/g]',
       'functional_groups', 'metal_linker', 'organic_linker1',
       'organic_linker2', 'topology', 'CO2/N2_selectivity',
       'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]',
       'CO2_working_capacity [mL/g]', 'CID', 'MolecularFormula',
       'MolecularWeight', 'InChI', 'InChIKey', 'IUPACName', 'XLogP',
       'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge',
       'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount',
       'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount',
       'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount',
       'UndefinedBondSter

In [ ]:
test_28.to_csv('test_28_final.csv',index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa57721c-b209-430d-97e4-796ac27ee623' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>